In [1]:
from experiments.ModelLoader import load_vae_model

# from google.colab import drive
# drive.mount('/content/drive')
# !git clone https://github.com/akshitv2/VAE-latent-space-experiment.git
# %cd VAE-latent-space-experiment
# !git checkout celebA-ParamTweaking
# !cp /content/drive/MyDrive/Datasets/img_align_celeba.zip /content/
# !unzip /content/img_align_celeba.zip -d /content/dataset > /dev/null
try:
    import google.colab
    running_in_colab = True
except ImportError:
    running_in_colab = False

if running_in_colab:
    dataset_dir = "/content/dataset"
    out_dir: str = "/content/drive/MyDrive/Temp/outputs"
    checkpoint_dir = "/content/drive/MyDrive/Temp/checkpoints"
else:
    dataset_dir = "G:/Temp"
    out_dir: str = "./outputs/"
    checkpoint_dir = "./experiments/checkpoints"

In [2]:
from torch import GradScaler
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import os
from tqdm import tqdm
from experiments.Checkpointing import save_checkpoint
from models.VAE import VAE
from modules.Losses import VAEVggLoss
from modules.SaveOutputs import save_reconstructions

torch.backends.cudnn.benchmark = True
torch.cuda.empty_cache()

dataset_dir: str = "./data/raw"
out_dir: str = "./outputs/"
batch_size: int = 64
latent_dim: int = 256
checkpoint_dir = "./experiments/checkpoints"
# epochs: int = 10
lr: float = 3e-4
beta: float = 0.5


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize((64, 64)),  # resize to 224x224
    transforms.ToTensor()  # convert to tensor & scale to [0,1]
])

dataset = datasets.ImageFolder(root="G:\Temp", transform=transform)
train_test_split_var = 0.99
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [int(train_test_split_var*len(dataset)), len(dataset) - int(train_test_split_var*len(dataset))])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=2, pin_memory=True)
n_train = len(train_loader.dataset)
print("Loaded datasets, number of samples: ", n_train)

# Model & Optimizer
scaler = GradScaler(device)
model = VAE(latent_dim=latent_dim).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

current_epoch = 1
os.makedirs(out_dir, exist_ok=True)

Loaded datasets, number of samples:  200573


In [3]:
# model = load_vae_model(checkpoint_path="./experiments/checkpoints/vae_checkpoint_epoch_16.pt", device=device, latent_dim=latent_dim)
# model.train()
# optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [22]:
epochs = 50
vgg_loss = VAEVggLoss(recon_weight=0.1, perc_weight=1.0, kl_weight=0.01, recon_loss_function = "mse")
for epoch in range(1, epochs + 1):
    model.train()
    running_total = running_recon = running_kld = running_perceptual =  0.0
    progress_bar = tqdm(enumerate(train_loader, start=1), total=len(train_loader), desc="Training")

    for batch_idx, (x, _) in progress_bar:
        x = x.to(device, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)
        logits, mean, logvar = model(x)
        loss, l1_loss, perc_loss, kl_loss = vgg_loss(logits, x, mean, logvar)
        # loss, l1_loss, perc_loss, kl_loss = criterion(x_recon, x, mu, logvar)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        progress_bar.set_postfix(
            loss=f"{loss.item():.3f}",
            l1=f"{l1_loss.item():.3f}",
            kld=f"{kl_loss.item():.3f}",
            percep=f"{perc_loss.item():.3f}",

        )
        running_total += loss.item()
        running_recon += l1_loss.item()
        running_kld += kl_loss.item()
        running_perceptual += perc_loss.item()
    current_epoch += 1
    print(
        f"Epoch {current_epoch:02d} | total: {running_total / n_train:.4f} | "
        f"recon: {running_recon / n_train:.4f} | kld: {running_kld / n_train:.4f} | "
        f"perceptual: {running_perceptual / n_train:.4f}"
    )
    save_reconstructions(model=model, x=x, out_dir=out_dir, step = current_epoch, device=device, variant="")
    # save_reconstructions(model, x, out_dir, current_epoch, device)
    if current_epoch % 10 == 0:
        save_checkpoint(model, optimizer, current_epoch, checkpoint_dir)

Training: 100%|██████████| 3134/3134 [06:27<00:00,  8.08it/s, kld=60.538, l1=61.115, loss=233.405, percep=111.751]


Epoch 14 | total: 3.7162 | recon: 0.9630 | kld: 0.9921 | perceptual: 1.7611


Training: 100%|██████████| 3134/3134 [08:07<00:00,  6.43it/s, kld=57.483, l1=63.732, loss=237.368, percep=116.152]


Epoch 15 | total: 3.6910 | recon: 0.9967 | kld: 0.9503 | perceptual: 1.7440


Training: 100%|██████████| 3134/3134 [06:00<00:00,  8.69it/s, kld=58.957, l1=60.613, loss=232.041, percep=112.471]


Epoch 16 | total: 3.6740 | recon: 1.0013 | kld: 0.9404 | perceptual: 1.7322


Training:  29%|██▉       | 921/3134 [01:45<04:14,  8.71it/s, kld=59.316, l1=65.984, loss=238.676, percep=113.376]


KeyboardInterrupt: 